# Big Data Analytics

## add names here:




### The aim of this project is to use the IMDB Dataset: (Link) and use data such as actors, director, genre, isAdult and runtime to predict how well a movie will perform. We use the boxoffice and userrating as measures of performance. We will train our model with 80% of the data and test it on the remaining 20%. We will also give our predicitons for upcoming movies that do not have any ratings/boxoffice yet.

### Our hypothesis is that actors & director will positively influence the performance while the variable runtime will negatively influence it. 



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.linear_model import Ridge,LinearRegression,Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline



# Load Data

In [ ]:




df = pd.read_csv("filename")


# extract train and test variables
data = df.to_numpy()
X = df.loc[:, df.columns != ''].to_numpy()   # enter column name of test variable
y = np.log(df['']).to_numpy()                # enter column name of test variable

print(X.shape)
print(y.shape)

In [36]:
# title.crew preprocessing
import os

print(os.listdir('.\data'))
df = pd.read_csv('.\data\data.tsv', sep='\t', header=0)
df.drop(['writers'], axis = 1, inplace=True)
print(df['directors'].isnull().values.any())
df

['data.tsv', 'README.md', 'title.crew.tsv']
False


,tconst,directors
0,tt0000001,nm0005690
1,tt0000002,nm0721526
2,tt0000003,nm0721526
3,tt0000004,nm0721526
4,tt0000005,nm0005690
...,...,...
7869181,tt9916848,"nm5519454,nm5519375"
7869182,tt9916850,"nm5519375,nm5519454"
7869183,tt9916852,"nm5519454,nm5519375"
7869184,tt9916856,nm10538645


# Data Preprocessing

## Steps performed

### dropping columns, imputing missing values, categorical datapoints


# Kfold to decrease potential overfitting

In [3]:
from sklearn.model_selection import KFold

def kfold_validation(X,y,model):
    start = time.time()
    kf = KFold(n_splits=5,shuffle=True, random_state=69420) 
    mses = []
    models = []
    count = 0
    for trainIndices,testIndices in kf.split(X):
        print(f"iteration: {count}")
        Xtrain,Xval = X[trainIndices,:],X[testIndices,:]
        ytrain,yval = y[trainIndices],y[testIndices]
        model.fit(Xtrain,ytrain)
        yhat = model.predict(Xval)
        mse = np.sum(np.square(yval-yhat))/yval.size # mean squared error
        mses.append(mse)
        models.append(model)
        count+=1
    print(f"time used (seconds): {time.time()-start}")
    return mses, models


# Standardise to make data more comparable

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Xtrain,Xtest, ytrain,ytest = train_test_split(X,y,shuffle=True,train_size=0.8)

def standardise_data(X):
    mean = np.mean(X)
    std = np.std(X)
    X_std = (X-mean)/std
    return X_std, mean, std

#kf = KFold(n_splits=1,shuffle=True, random_state=69420) 

Xtrain_std, X_train_mean, X_train_std_div = standardise_data(Xtrain)
Xtest_std = (Xtest-X_train_mean)/X_train_std_div


NameError: name 'X' is not defined

# Linear Regression

In [5]:
model = LinearRegression()
mses,models = kfold_validation(Xtrain,ytrain,model)
yhat=models[np.argmin(mses)].predict(Xtest)
np.mean(np.square(yhat-ytest))

NameError: name 'Xtrain' is not defined

# Random Forest

In [6]:
from sklearn.ensemble import RandomForestRegressor 

model = RandomForestRegressor()
mses,models = kfold_validation(Xtrain,ytrain,model)
yhat=models[np.argmin(mses)].predict(Xtest)
np.mean(np.square(yhat-ytest))


NameError: name 'Xtrain' is not defined

# Data Visualisation